<a href="https://colab.research.google.com/github/abegpatel/movie-recomendation-system-using-Autoencoder/blob/master/boltzman_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**boltzman machine:**
1.boltzman machine
2.energy based model(EBM)
3.restricted Boltzman machine(RBM)
4.contrastive divergence(CD)
5.deep boltzman network(DBN)



**Energy based model**
.it is aunsupervised learning
.used for recomendation system
.visible node(input node)->hidden nodes(hidden layer)
.evry input connect to evrey hidden layer
.bi-directional
.capabel generating parameters of its own->doesnot require i/p
.generates diff states
.stocastic or generative deep learn model
.training data->boltzman machine->adjust weights(learn foem each other charcterstics)parameter interact with each other
.all parameter connected to each other(not visible not hidden)





**energy based model**(EBM)
.pi=(e^-energyi /kT)/(sum j=1 to M)e^-energyj/kT
.the probability of state inversely proportional to energy of the sys
eg.room has wind molecule
 energy is high of molecule the probability is low(if for all molecule are placed together in corner)
 if any thing like water,molecule,ink(it can be spread every where)becauce that is their lowest energy state..)always these element are find its lower energy state
.energy is define through its weights..and it try to get its lowest energy state
E(v,h)=-sumi(aivi)-sumj(bjhj)-sumisumj(viwihj)
p(v,h)=1/Ze^-e(v,h)#probability




**.restricted Boltzman machine(RBM)**
.here hidden nodes can not connected to each other
and visible nodes connected to each other
.learn how to allocate the hidden nodes to certain features
.finding som association rules->connect to a certain features
if new row->if like (1) unlike(0)
(train it self to get the certain feature connceted to hidden attributs
.to make an prediction
.acquiring certaion features from hidden layer and predict unkown visible layer



**contrastive divergence(CD)**
how weights are adjusted
(weights are same) equal weights->reconstruct the i/p
(sampling)
sample towards the lowest energy state-> creates a model
.energy curve







**5.deep belief network(DBN)**
 .greedy layer wise train algorithm-(rbm)
 .wake-sleep algo-(all up then all down)
 



**6.deep botlthman machine**
can extract complex features


In [ ]:
!unzip -uq "/content/drive/My Drive/Colab Notebooks/P16-Boltzmann-Machines.zip" -d "/content/drive/My Drive/"

In [ ]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import pandas as pd
from torch.autograd import Variable
import numpy as np

In [ ]:
# Importing the dataset
movies = pd.read_csv('/content/drive/My Drive/Boltzmann_Machines/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('/content/drive/My Drive/Boltzmann_Machines/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('/content/drive/My Drive/Boltzmann_Machines/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [ ]:
# Preparing the training set and the test set
training_set = pd.read_csv('/content/drive/My Drive/Boltzmann_Machines/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('/content/drive/My Drive/Boltzmann_Machines/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [ ]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))


In [ ]:
print(nb_users)
print(nb_movies)

943
1682


In [ ]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)


In [ ]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [ ]:
# Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1



In [ ]:
# Creating the architecture of the Neural Network
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)



In [ ]:
# Training the RBM
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))



epoch: 1 loss: tensor(0.0586)
epoch: 2 loss: tensor(0.)
epoch: 3 loss: tensor(0.)
epoch: 4 loss: tensor(0.)
epoch: 5 loss: tensor(0.)
epoch: 6 loss: tensor(0.)
epoch: 7 loss: tensor(0.)
epoch: 8 loss: tensor(0.)
epoch: 9 loss: tensor(0.)
epoch: 10 loss: tensor(0.)


In [ ]:
# Testing the RBM
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.)
